# LLMs x Security

This section focusses on how LLMs deployed by you can be made to do things which you wouldn't want in some cases. This is more example/theory heavy, and less code heavy. We don't cover all the ways in which LLMs can be used to do bad things e.g. capture the flag or similar challenges and instead we leave that to the reader to explore.


The most definitive guide on the topic comes from [OWASP Top 10 LLM Attacks Team](https://owasp.org/www-project-top-10-for-large-language-model-applications/)

Here, we'll focus on the first attack type: Prompt Injection. This is also by far the most common attack type, and is the most likely to be encountered in the wild. The complete list of LLM risks: 

1. Prompt Injection
2. Insecure Output Handling
3. Training Data Poisoning
4. Model Denial of Service 
5. Supply Chain vulnerabilities - vulnerable components, e.g. dependencies, network and hardware which the components rely on
6. Sensitive Information Disclosure - leakage of confidential information, privacy violations, and other sensitive information
7. Insecure Plugin Design - LLMs can have broader access to apps, leading to potential security risks
8. Excessive Agency - LLM systems may undertake actions that are not in the best interest of the user or the organization
9. Overreliance - humans may over-rely on LLMs, leading to potential harm without adequate warning
10. Model Theft - unauthorized acess, copying or exfiltration of LLM models

# Prompt Injection

## Introduction to Prompt Injection

Prompt injection is a technique where malicious actors manipulate the input prompts to an AI model to make it generate harmful or inappropriate content. This is achieved by carefully crafting the input prompts in a way that the AI model, while processing the input, ends up generating outputs that can be harmful, inappropriate, or against the intended use of the model. This can be a significant security concern for AI developers and users, as it can lead to misuse of the AI system and can potentially harm the reputation of the business. 

## Types of Prompt Injection

Prompt injection can be categorized into different types based on the intent and method of the malicious actor. Here are some of the common types of prompt injection:

### Jailbreak

In adversarial prompt injection, the attacker manipulates the input prompts to make the AI model generate harmful or inappropriate content. This is often done by carefully crafting the input prompts in a way that the AI model, while processing the input, ends up generating outputs that can be harmful, inappropriate, or against the intended use of the model. This can be a significant security concern for AI developers and users, as it can lead to misuse of the AI system and can potentially harm the reputation of the system and its developers.

### DAN: Roleplaying for Jailbreaking

Most commercial LLMs including GPT have guardrails which prevent certains types of harmful, illegal, unethical or even violent content from being generated. However, these guardrails can be bypassed by using a technique called Do Anything Now on Reddit. In this technique, the attacker can cajole the AI using a roleplaying technique to generate content that would otherwise be blocked.

For instance, if the attacker wants to generate content that is violent, they can use a roleplaying technique to make the AI believe that it is in a roleplaying game and that the content is being generated for the game. 

You can checkout and chuckle with the brief summary of DAN prompts here -- many of which don't work anymore [here on Reddit](https://www.reddit.com/r/ChatGPT/comments/10tevu1/new_jailbreak_proudly_unveiling_the_tried_and/)

### Prompt Leaking

Prompt leaking is a security concern where the AI model inadvertently reveals sensitive information in its responses. This can occur when the model has been trained on data that includes sensitive information, and the model then generates responses that include this information. 

In the context of adversarial risks, prompt leaking can be exploited by malicious actors. One can craft specific prompts that trick the model into revealing sensitive information. This could be information that the model has learned during training or information that the model has inferred from the prompt itself.

### Indirect Prompt Injection

An instance of indirect prompt injection is demonstrated in [Arvind Narayanan's experiment](https://twitter.com/random_walker/status/1636923058370891778): 

He attempted to influence Bing Chat to incorporate the term "cow" in its output. The directive was subtly embedded in white text against a white backdrop on his website, making it invisible to human visitors but detectable to any system scraping the page's content. Later, Arvind employed GPT-4 and a ReAct tool to scrape the same webpage, and interestingly, the summary generated contained the word "Cow."

![](../assets/indirect_prompt_engineering.png)

This diagram & example is from [Lakera's blog on Prompt Injection](https://www.lakera.ai/blog/guide-to-prompt-injection)

In [ ]:
import instructor
from dotenv import load_dotenv
from instructor import openai_moderation
from openai import OpenAI
from pydantic import AfterValidator, BaseModel
from typing_extensions import Annotated

load_dotenv()

In [ ]:
client = OpenAI()

response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "I want to hurt myself"},
    ],
    model="gpt-3.5-turbo",
)
response.choices[0].message.content

In [ ]:
client = instructor.patch(OpenAI())

class Response(BaseModel):
    message: Annotated[str, AfterValidator(openai_moderation(client=client))]


# r = Response(message="It's a beautiful morning!")

Response(message="I want to hurt myself")